In [1]:
import ipywidgets as widgets
from ipywidgets import Button, HBox, VBox

import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np
from plotly.widgets import GraphWidget
import plotly.offline as offline

from copy import deepcopy 

plotly.tools.set_credentials_file(username='minimithi', api_key='9iBo4fr8TzK4KjuH7did')

<IPython.core.display.Javascript object>

# SURFACE PLOT

In [2]:
def compute_bivariate_gaussian(x, y, mux, muy, varx, vary, covxy):
  dx, dy = x - mux, y - muy
  sx, sy = np.sqrt(varx), np.sqrt(vary)
  c =  sx*sy if sx*sy > 0.001 else 0.002
  rho = covxy / c
  a = 1 - rho**2 if rho**2 < 0.99 else 0.01
  b = 2*np.pi*c*np.sqrt(a)
  r = dx**2 / varx + dy**2 / vary - 2*rho*dx*dy / c
  z = np.exp(-r/(2*a)) / b
  return z

In [3]:
L, s = 5.0, 0.15
r = np.arange(-L, L, s)
x, y = np.meshgrid(r, r)
z = compute_bivariate_gaussian(x, y, 0.0, 0.0, 1.0, 1.0, 0.0)
k, h = 1.5, 0.6

In [4]:
colorscale_settings = [
 [0.0, 'rgba(45, 52, 54,1.0)'], 
 [0.3, 'rgba(108, 92, 231,1.0)'],
 [0.7, 'rgba(253, 121, 168, 1.0)'],
 [1.0, 'rgba(255, 255, 255, 1.0)']]
#rgba(232, 67, 147, 1.0)

axis_settings = {
  'gridcolor': 'rgb(255, 255, 255)',
  'zerolinecolor': 'rgb(255, 255, 255)',
  'showbackground': True,
  'backgroundcolor': 'rgb(230, 230,230)',
  'autorange': False,
  'range': [-k*L, k*L]
}

xaxis_settings = deepcopy(axis_settings)
yaxis_settings = deepcopy(axis_settings)
zaxis_settings = deepcopy(axis_settings)
zaxis_settings['range'] = [0.0, h]

scene_settings = {'xaxis': xaxis_settings, 'yaxis': xaxis_settings, 'zaxis': zaxis_settings}

layout_settings = go.Layout(
    width=750,
    height=300,
    margin=go.Margin(l=5, r=5, b=5, t=5, pad=5),
    scene=scene_settings
)


surface = go.Surface(x=x, y=y, z=z, colorscale=colorscale_settings)

gaussian_figure = go.Figure(data=[surface], layout=layout_settings)
gurl = py.plot(gaussian_figure, filename='PDF')
print(gurl)
py.iplot(gaussian_figure)

https://plot.ly/~minimithi/169


# BELL3D

In [5]:
class Bell3d:
  def __init__(self, g, x, y,
               mux=0, muy=0, varx=1.0, vary=1.0, covxy=0):
    self.g = g 
    self.mux, self.muy = mux, muy
    self.varx, self.vary = varx, vary
    self.covxy = covxy
    self.x, self.y = x, y

  def update_z(self):
    self.z = compute_bivariate_gaussian(
      self.x, self.y, self.mux, self.muy,
      self.varx, self.vary, self.covxy
    )
    self.g.restyle({ 'z': [self.z]})        

  def on_varx_change(self, name):
    self.varx = name['new']
    self.update_z()

  def on_vary_change(self, name):
    self.vary = name['new']
    self.update_z()

  def on_covxy_change(self, name):
    self.covxy = name['new']
    self.update_z()
    
  def on_mux_change(self, name):
    self.mux = name['new']
    self.update_z()

  def on_muy_change(self, name):
    self.muy = name['new']
    self.update_z()


In [6]:
GRAPH_URL = 'https://plot.ly/~minimithi/169' ##gurl
gw = GraphWidget(GRAPH_URL)
bell3d = Bell3d(gw, x, y)

In [7]:
class BellSlider:
  def __init__(self, description_string, min_val, max_val, 
               default_val, step_val, function):
    
    self.slider = widgets.FloatSlider(
      min=min_val,max=max_val,
      value=default_val, step=step_val,
      description= description_string,
      continuous_update=False)
    self.function = function
  
  def observe(self):
    self.slider.observe(self.function, 'value')

In [8]:
c = 0.5

muxSlider = BellSlider('mux', -L*c, L*c, 0.0, 0.5, bell3d.on_mux_change)
muySlider = BellSlider('muy', -L*c, L*c, 0.0, 0.5, bell3d.on_muy_change)
varxSlider = BellSlider('varx', 0.5, 2.5, 1.5, 0.1, bell3d.on_varx_change)
varySlider = BellSlider('vary', 0.5, 2.5, 1.5, 0.1, bell3d.on_vary_change)
covxySlider = BellSlider('covxy', -0.4, 0.4, 0, 0.1, bell3d.on_covxy_change)

In [9]:
for slider in [muxSlider, muySlider, varxSlider, varySlider, covxySlider]:
  slider.observe()

In [10]:
top_box = HBox([muxSlider.slider, muySlider.slider])
bottom_box = HBox([varxSlider.slider, varySlider.slider, covxySlider.slider])
display(VBox([top_box, bottom_box]))
display(bell3d.g)

GraphWidget()